In [58]:
# dependencies

import pandas as pd
from sqlalchemy import create_engine

## Further data clean up

In [59]:
# file paths

weather_file_1 = 'Resources/avg_min_temp.csv'
weather_file_2 = 'Resources/avg_max_temp.csv'
population_file = 'Resources/StatePopulation.csv'
smoking_file = 'Resources/New_Tobacco.csv'

In [75]:
# store csv's into dataframes

weather_min_df = pd.read_csv(weather_file_1)
weather_max_df = pd.read_csv(weather_file_2)
population_df = pd.read_csv(population_file)
smoking_df = pd.read_csv(smoking_file)

In [99]:
weather_min_df.head()

State  Avg Min Temp (°F)
0       Hawaii              65.09
1      Florida              61.50
2    Louisiana              56.07
3        Texas              54.16
4  Mississippi              52.26

In [100]:
# renaming headers to match sql database

weather_min_df = weather_min_df.rename(columns={"State": "state", "Avg Min Temp (°F)": "avg_min_temp"})
weather_min_df.head()

state  avg_min_temp
0       Hawaii         65.09
1      Florida         61.50
2    Louisiana         56.07
3        Texas         54.16
4  Mississippi         52.26

In [101]:
weather_max_df = weather_max_df.rename(columns={"State": "state", "Avg Max Temp (°F)": "avg_max_temp"})
weather_max_df.head()

state  avg_max_temp
0    Florida         81.97
1     Hawaii         80.66
2    Arizona         80.59
3      Texas         77.73
4  Louisiana         77.33

In [103]:
# joining weather max and min to make one DF

weather_df = pd.merge(weather_min_df, weather_max_df, on='state')
weather_df.head()

state  avg_min_temp  avg_max_temp
0       Hawaii         65.09         80.66
1      Florida         61.50         81.97
2    Louisiana         56.07         77.33
3        Texas         54.16         77.73
4  Mississippi         52.26         74.67

In [104]:
weather_df.dtypes

state            object
avg_min_temp    float64
avg_max_temp    float64
dtype: object

In [90]:
population_df.head()

State  Population
0     Alabama       17464
1      Alaska        1746
2     Arizona       60013
3    Arkansas       12256
4  California       56627

In [91]:
# renaming headers to match sql database

population_df = population_df.rename(columns={"State": "state", "Population": "population"})
population_df.head()

state  population
0     Alabama       17464
1      Alaska        1746
2     Arizona       60013
3    Arkansas       12256
4  California       56627

In [105]:
population_df.dtypes

state         object
population     int64
dtype: object

In [76]:
smoking_df.head()

State Smoke everyday Smoke some days Former smoker Never smoked
0  Puerto Rico          7.50%           4.40%        17.30%       70.80%
1       Nevada         16.50%           4.90%        25.80%       52.80%
2    Louisiana         15.90%           6.20%           22%          56%
3   California          7.50%           4.60%        23.10%       64.80%
4       Kansas         11.90%           5.10%        24.20%       58.80%

In [78]:
# split % from smoke values 

split_1 = smoking_df['Smoke everyday'].str.split('%', n=1, expand=True)
split_2 = smoking_df['Smoke some days'].str.split('%', n=1, expand=True)
split_3 = smoking_df['Former smoker'].str.split('%', n=1, expand=True)
split_4 = smoking_df['Never smoked'].str.split('%', n=1, expand=True)


smoking_df['smoke_everyday'] = split_1[0]
smoking_df['smoke_somedays'] = split_2[0]
smoking_df['smoke_former'] = split_3[0]
smoking_df['smoke_never'] = split_4[0]


smoking_df.drop(columns=['Smoke everyday'], inplace=True)
smoking_df.drop(columns=['Smoke some days'], inplace=True)
smoking_df.drop(columns=['Former smoker'], inplace=True)
smoking_df.drop(columns=['Never smoked'], inplace=True)

smoking_df.head()

State smoke_everyday smoke_somedays smoke_former smoke_never
0  Puerto Rico           7.50           4.40        17.30       70.80
1       Nevada          16.50           4.90        25.80       52.80
2    Louisiana          15.90           6.20           22          56
3   California           7.50           4.60        23.10       64.80
4       Kansas          11.90           5.10        24.20       58.80

In [79]:
# renaming headers to match sql database

smoking_df = smoking_df.rename(columns={"State": "state"})
smoking_df.head()

state smoke_everyday smoke_somedays smoke_former smoke_never
0  Puerto Rico           7.50           4.40        17.30       70.80
1       Nevada          16.50           4.90        25.80       52.80
2    Louisiana          15.90           6.20           22          56
3   California           7.50           4.60        23.10       64.80
4       Kansas          11.90           5.10        24.20       58.80

In [80]:
smoking_df.dtypes

state             object
smoke_everyday    object
smoke_somedays    object
smoke_former      object
smoke_never       object
dtype: object

In [81]:
smoking_df['smoke_everyday'] = pd.to_numeric(smoking_df['smoke_everyday'])

In [82]:
smoking_df['smoke_somedays'] = pd.to_numeric(smoking_df['smoke_somedays'])

In [84]:
smoking_df['smoke_former'] = pd.to_numeric(smoking_df['smoke_former'])

In [85]:
smoking_df['smoke_never'] = pd.to_numeric(smoking_df['smoke_never'])

In [86]:
smoking_df.dtypes

state              object
smoke_everyday    float64
smoke_somedays    float64
smoke_former      float64
smoke_never       float64
dtype: object

In [87]:
smoking_states = smoking_df["state"].unique()
smoking_states

array(['Puerto Rico', 'Nevada', 'Louisiana', 'California', 'Kansas',
       'Massachusetts', 'Arkansas', 'New Jersey', 'New York', 'Arizona',
       'Vermont', 'Alabama', 'Idaho', 'Wisconsin', 'Virginia', 'Michigan',
       'Ohio', 'Illinois', 'Connecticut', 'Maine', 'Iowa',
       'North Carolina', 'Colorado', 'Utah', 'Tennessee', 'Oklahoma',
       'Indiana', 'Wyoming', 'Florida', 'West Virginia', 'South Carolina',
       'Oregon', 'New Hampshire', 'Delaware', 'Hawaii', 'Texas',
       'Minnesota', 'Pennsylvania', 'New Mexico', 'Rhode Island',
       'Alaska', 'Missouri', 'Washington', 'Georgia', 'Nebraska',
       'North Dakota', 'District of Columbia', 'South Dakota',
       'Mississippi', 'Montana', 'Kentucky', 'Maryland',
       'Nationwide (States, DC, and Territories)',
       'Nationwide (States and DC)', 'Guam', 'Virgin Islands'],
      dtype=object)

In [92]:
population_states = population_df["state"].unique()
population_states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [93]:
len(smoking_states)

56

In [94]:
len(population_states)

51

In [95]:
# create new smoking table with inner join on population table so that states match

new_smoking_df = pd.merge(smoking_df, population_df, on='state')
new_smoking_df.head()


state  smoke_everyday  smoke_somedays  smoke_former  smoke_never  \
0         Nevada            16.5             4.9          25.8         52.8   
1      Louisiana            15.9             6.2          22.0         56.0   
2     California             7.5             4.6          23.1         64.8   
3         Kansas            11.9             5.1          24.2         58.8   
4  Massachusetts            10.2             3.9          29.3         56.6   

   population  
0       55274  
1        9616  
2       56627  
3          31  
4      181045

In [96]:
new_smoking_states = new_smoking_df["state"].unique()
len(new_smoking_states)

51

In [97]:
new_smoking_df = new_smoking_df[['state', 'smoke_everyday', 'smoke_somedays', 'smoke_former', 'smoke_never']]
new_smoking_df.head()

state  smoke_everyday  smoke_somedays  smoke_former  smoke_never
0         Nevada            16.5             4.9          25.8         52.8
1      Louisiana            15.9             6.2          22.0         56.0
2     California             7.5             4.6          23.1         64.8
3         Kansas            11.9             5.1          24.2         58.8
4  Massachusetts            10.2             3.9          29.3         56.6

## Uploading to PGAdmin

In [114]:
# connect to local database

rds_connection_string = "usr1:password@localhost:5432/final_smoking_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [115]:
# check for tables

engine.table_names()

['state_population', 'state_weather', 'state_smoking']

In [116]:
# use pandas to load dataframes onto smoking_db

population_df.to_sql(name='state_population', con=engine, if_exists='append', index=False)

In [117]:
pd.read_sql_query('select * from state_population', con=engine).head()

state  population
0     Alabama       17464
1      Alaska        1746
2     Arizona       60013
3    Arkansas       12256
4  California       56627

In [118]:
weather_df.to_sql(name='state_weather', con=engine, if_exists='append', index=False)

In [119]:
pd.read_sql_query('select * from state_weather', con=engine).head()

state  avg_min_temp  avg_max_temp
0       Hawaii         65.09         80.66
1      Florida         61.50         81.97
2    Louisiana         56.07         77.33
3        Texas         54.16         77.73
4  Mississippi         52.26         74.67

In [120]:
new_smoking_df.to_sql(name='state_smoking', con=engine, if_exists='append', index=False)

In [121]:
pd.read_sql_query('select * from state_smoking', con=engine).head()

state  smoke_everyday  smoke_somedays  smoke_former  smoke_never
0         Nevada            16.5             4.9          25.8         52.8
1      Louisiana            15.9             6.2          22.0         56.0
2     California             7.5             4.6          23.1         64.8
3         Kansas            11.9             5.1          24.2         58.8
4  Massachusetts            10.2             3.9          29.3         56.6

## Load Database

In [124]:
# load table containing all all rows from 3 tables joined on State

pd.read_sql_query('select p.state, p.population, w.avg_min_temp, w.avg_max_temp,\
    s.smoke_everyday, s.smoke_somedays, s.smoke_former, s.smoke_never\
    from state_population as p\
    left join state_weather as w on p.state=w.state\
    left join state_smoking as s on w.state=s.state',con=engine).head(51)

state  population  avg_min_temp  avg_max_temp  \
0                 Hawaii      154834         65.09         80.66   
1                Florida       19540         61.50         81.97   
2              Louisiana        9616         56.07         77.33   
3                  Texas        4539         54.16         77.73   
4            Mississippi       16064         52.26         74.67   
5                Arizona       60013         51.31         80.59   
6                Alabama       17464         51.01         74.31   
7                Georgia       13790         50.69         73.90   
8         South Carolina      127424         49.82         73.58   
9             California       56627         48.97         73.36   
10              Arkansas       12256         48.72         71.42   
11              Oklahoma        6314         48.17         72.04   
12        North Carolina       16125         47.21         70.16   
13             Tennessee       62655         46.90         69.30   
14  District of Columbia      601723         44.54         65.02   
15              Virginia       26203         44.26         66.77   
16              Kentucky       14730         44.25         66.98   
17              Delaware      148567         44.21         64.62   
18              Maryland      620961         44.19         65.08   
19              Missouri      319294         43.84         65.53   
20                Nevada       55274         42.75         71.64   
21                Kansas          31         42.72         66.58   
22               Indiana        1281         41.65         61.91   
23         West Virginia       20175         41.61         64.51   
24            New Jersey        4882         41.45         62.40   
25              Illinois        5274         41.18         61.51   
26                  Ohio        1124         40.79         60.94   
27            Washington       83889         40.76         60.09   
28                Oregon       23546         40.11         62.36   
29          Pennsylvania         709         39.29         60.31   
30          Rhode Island       22787         39.02         59.56   
31              New York         974         38.34         58.01   
32           Connecticut        7964         37.95         59.27   
33              Nebraska        3441         37.82         61.91   
34                  Iowa        2967         37.67         58.57   
35         Massachusetts      181045         37.63         58.61   
36            New Mexico       46901         37.46         68.66   
37              Michigan        1072         36.73         56.48   
38                  Utah       14074         36.27         62.85   
39             Wisconsin         796         34.44         54.96   
40          South Dakota        1479         33.78         57.26   
41                 Idaho        4192         33.29         59.31   
42         New Hampshire        1123         33.23         55.69   
43               Vermont        2148         32.96         54.48   
44                 Maine       12529         32.53         53.65   
45             Minnesota         231         32.49         53.41   
46              Colorado        3920         31.65         60.84   
47               Montana       36892         31.05         57.27   
48          North Dakota          24         29.38         52.13   
49               Wyoming        2576         29.22         57.63   
50                Alaska        1746         24.23         39.80   

    smoke_everyday  smoke_somedays  smoke_former  smoke_never  
0             10.7             3.8          25.3         60.2  
1             12.0             5.2          29.8         53.0  
2             15.9             6.2          22.0         56.0  
3             10.5             5.3          21.3         62.9  
4             17.1             5.8          22.0         55.0  
5             10.7             4.4          27.9         57.1  
6             15.6             6.3